<a href="https://colab.research.google.com/github/channacy/Whats-That-Rash/blob/main/Fine_Tuning_GPT_With_Skin_Condition_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources

https://platform.openai.com/docs/guides/vision-fine-tuning

https://www.datacamp.com/tutorial/fine-tuning-openais-gpt-4-step-by-step-guide

Dataset:

https://www.kaggle.com/datasets/riyaelizashaju/skin-disease-classification-image-dataset

In [26]:
from google.colab import userdata, drive
from openai import OpenAI

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import base64

def encode_image(image):
    return base64.b64encode(image.read()).decode("utf-8")

IMAGE_PATH = "/content/drive/MyDrive/Skin_Images/train"
data_for_finetuning = []


for root, dirs, files in os.walk(IMAGE_PATH):
    for dir_name in dirs:
        skin_condition = dir_name
        print(f"Processing directory: {skin_condition}")
        dir_path = os.path.join(root, dir_name)
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "rb") as image_file:
                        encoded_image = encode_image(image_file)
                        # Construct the object
                        data_object = {
                          "messages": [
                            {
                              "role": "system",
                              "content": "You are an assistant that identifies skin conditions and diseases."
                            },
                            {
                              "role": "user",
                              "content": "What is this skin condition?"
                            },
                            {
                              "role": "user",
                              "content": [
                                {
                                  "type": "image_url",
                                  "image_url": {
                                    "url": f"data:image/jpeg;base64,{encoded_image}"
                                  }
                                }
                              ]
                            },
                            {
                              "role": "assistant",
                              "content": skin_condition
                            }
                          ]
                        }
                        data_for_finetuning.append(data_object)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Now 'data_for_finetuning' contains the list of objects for fine-tuning

Processing directory: Dermatofibroma
Processing directory: Atopic Dermatitis
Processing directory: Vascular lesion
Processing directory: Squamous cell carcinoma
Processing directory: Actinic keratosis
Processing directory: Benign keratosis
Processing directory: Tinea Ringworm Candidiasis
Processing directory: Melanoma
Processing directory: Melanocytic nevus


In [ ]:
# Specify the index of the example to remove (112th example means index 111)
index_to_remove = 111

# Remove the example at the specified index
if index_to_remove < len(data_for_finetuning):
    removed_item = data_for_finetuning.pop(index_to_remove)
    print(f"Removed example at index {index_to_remove} (112th example) due to potential usage policy violation.")
else:
    print(f"Index {index_to_remove} is out of bounds. No item removed.")


Removed example at index 111 (112th example) due to potential usage policy violation.


In [ ]:
len(data_for_finetuning)

696

In [ ]:
import json

# Assuming 'data_for_finetuning' is already populated from the previous step

output_filename = "train.jsonl"

with open(output_filename, "w") as outfile:
  count = 0
  for entry in data_for_finetuning:
      json.dump(entry, outfile)
      outfile.write('\n')
      count += 1
      if count >= 1000:
        break

print(f"Successfully created {output_filename}")


Successfully created train.jsonl


In [ ]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
client.files.create(
  file=open("train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-JNzXAkhYLQXBFp3WeLqhtV', bytes=190837896, created_at=1747609851, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [ ]:
len(data_for_finetuning)

696

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-JNzXAkhYLQXBFp3WeLqhtV",
  model="gpt-4o-2024-08-06"
)

FineTuningJob(id='ftjob-ny6pKLPz4kOgNINUKu5wQjpJ', created_at=1747609870, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-L5HXmW8uxG8wCHFJ7HrXvLMY', result_files=[], seed=1033442840, status='validating_files', trained_tokens=None, training_file='file-JNzXAkhYLQXBFp3WeLqhtV', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)